In [ ]:
!unzip drive/My\ Drive/CompleteDataset.zip -d dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import keras as kr
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import MaxPooling2D,Dense,Conv2D,Flatten,Dropout
import tensorboard

import os,datetime


Using TensorFlow backend.


In [10]:
train_dir = 'drive/My Drive/Fruits360/Dataset/Train_set'
valid_dir = 'drive/My Drive/Fruits360/Dataset/Validation_set'

In [11]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(100, 100),
        class_mode='categorical',
        batch_size = 32)
validation_generator = test_datagen.flow_from_directory(
        valid_dir,
        target_size=(100, 100),
        class_mode='categorical',
        batch_size = 32)


Found 10597 images belonging to 18 classes.
Found 4987 images belonging to 18 classes.


In [14]:
train_generator.class_indices

{'APPLES': 0,
 'BANANA': 1,
 'BLUEBERRY': 2,
 'CHERRY': 3,
 'DATES': 4,
 'GRAPES': 5,
 'GUAVA': 6,
 'LEMON': 7,
 'MANGO': 8,
 'ONION': 9,
 'ORANGE': 10,
 'PAPAYA': 11,
 'PINEAPPLE': 12,
 'POMEGRANATE': 13,
 'POTATO': 14,
 'STRAWBERRY': 15,
 'TOMATO': 16,
 'WATERMELON': 17}

## Creating Model

In [ ]:

model = tf.keras.Sequential([Conv2D(filters = 18,kernel_size=(3,3),activation='relu'),
                            MaxPooling2D((2,2)),
                            Dropout(0.5),
                             
                            Conv2D(filters =36,kernel_size=(3,3),activation='relu'),
                            MaxPooling2D((2,2)),
                            Dropout(0.5),
                             
                            Conv2D(filters = 72,kernel_size=(3,3),activation='relu'),
                            Dropout(0.5),
                             
                            Conv2D(filters = 288,kernel_size=(3,3),activation='relu'),
                            MaxPooling2D((2,2)),
                            
                            Flatten(),
                             
                            Dense(72,activation='relu'),
                            Dense(18,activation='softmax')])

model.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)
model.build((None,100,100,3))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            multiple                  504       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 multiple                  0         
_________________________________________________________________
dropout_6 (Dropout)          multiple                  0         
_________________________________________________________________
conv2d_9 (Conv2D)            multiple                  5868      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 multiple                  0         
_________________________________________________________________
dropout_7 (Dropout)          multiple                  0         
_________________________________________________________________
conv2d_10 (Conv2D)           multiple                 

In [ ]:
NUM_EPOCHS = 100 #@param {type:'slider',min:3,max:100}

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=3)
logdir="drive/My Drive/Fruits360/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)


In [ ]:
%%time
model.fit(
        train_generator,
        epochs = NUM_EPOCHS,
        validation_data=validation_generator[0],
        validation_freq=1,
        verbose =1,
        callbacks=[early_stopping,tensorboard_callback]
    )

Epoch 1/100
332/332 [==============================] - 39s 117ms/step - loss: 1.6103 - accuracy: 0.5018 - val_loss: 2.2096 - val_accuracy: 0.4688
Epoch 2/100
332/332 [==============================] - 39s 117ms/step - loss: 0.2529 - accuracy: 0.9176 - val_loss: 2.0381 - val_accuracy: 0.6562
Epoch 3/100
332/332 [==============================] - 39s 116ms/step - loss: 0.0978 - accuracy: 0.9677 - val_loss: 2.7937 - val_accuracy: 0.7500
Epoch 4/100
332/332 [==============================] - 38s 115ms/step - loss: 0.0816 - accuracy: 0.9762 - val_loss: 3.0973 - val_accuracy: 0.7500
Epoch 5/100
332/332 [==============================] - 39s 117ms/step - loss: 0.0409 - accuracy: 0.9866 - val_loss: 3.1409 - val_accuracy: 0.8438
Epoch 6/100
332/332 [==============================] - 39s 117ms/step - loss: 0.0467 - accuracy: 0.9883 - val_loss: 2.0155 - val_accuracy: 0.8438
Epoch 7/100
332/332 [==============================] - 39s 117ms/step - loss: 0.0453 - accuracy: 0.9859 - val_loss: 2.5559 -

In [ ]:
def save_model(model,suffix=None):
    modeldir = os.path.join("drive/My Drive/Fruits360/models",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    model_path = modeldir + "_"+ suffix+".h5"
    model.save(model_path)
    return(model_path)
save_model(model,"18-cat-allimages-adam-0.84")

In [ ]:
model.evaluate(validation_generator)

156/156 [==============================] - 1926s 12s/step - loss: 1.8522 - accuracy: 0.7945


[1.8522005081176758, 0.7944656014442444]